In [1]:
import sys

sys.path.append("..")
import ml_utils.utils as ml_utils
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import tensorflow as tf
from keras.metrics import Precision, Recall
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    LSTM,
    Dense,
    Embedding,
    Bidirectional,
    Dropout,
    GlobalMaxPooling1D,
    Concatenate,
    BatchNormalization,
    Conv1D,
    ReLU,
    Input,
    GRU,
)
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras.layers import Embedding, Flatten, Dense, Attention
import tensorflow_datasets as tfds
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, pos_tag, ne_chunk
import os
from keras.preprocessing.sequence import TimeseriesGenerator
from datetime import datetime
from datasets import load_dataset
import json
from nltk.tokenize import word_tokenize
from sklearn.calibration import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

2024-03-17 18:36:34.249307: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 18:36:34.249375: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 18:36:34.251573: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-17 18:36:34.264142: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-17 18:36:35.683072: W tensorflow/compiler/tf2

In [13]:
# np_train = np.load(
#     "../datasets/COQA-Conversation/COQA_train_final.npz", allow_pickle=True
# )
np_test = np.load(
    "../datasets/COQA-Conversation/COQA_test_final.npz", allow_pickle=True
)
test_df = pd.DataFrame(np_test["data"], columns=np_test["columns"])
# train_df = pd.DataFrame(np_train["data"], columns=np_train["columns"])
# np_train = None
# np_test = None

In [37]:
new_rows = ml_utils.extract_each_value_from_column(test_df)
# Create a new DataFrame from the list of new rows
test_df = pd.DataFrame(new_rows)

new_rows = ml_utils.extract_each_value_from_column(train_df)
# Create a new DataFrame from the list of new rows
train_df = pd.DataFrame(new_rows)


test_df = test_df.map(ml_utils.remove_stopwords)

test_df = test_df.map(ml_utils.stem_text)

train_df = train_df.map(ml_utils.remove_stopwords)

train_df = train_df.map(ml_utils.stem_text)


def tokenize_sentences_by_columns(df, column):
    tr_text = df[column]
    tokenizer = Tokenizer(num_words=10000)
    tokenizer.fit_on_texts(tr_text)

    sequences = tokenizer.texts_to_sequences(tr_text)
    return sequences


def pad_sequences_with_zeros(X, maxlen):
    X = X.tolist()
    return pad_sequences(X, maxlen=maxlen).tolist()


test_tfidf = pd.DataFrame()
train_tfidf = pd.DataFrame()

test_padded = pd.DataFrame()
train_padded = pd.DataFrame()

for col in test_df.columns:
    test_tfidf[col] = tokenize_sentences_by_columns(test_df, col)
for col in train_df.columns:
    train_tfidf[col] = tokenize_sentences_by_columns(train_df, col)


for col in test_tfidf.columns:
    test_padded[col] = pad_sequences_with_zeros(test_tfidf[col], 1000)
for col in train_tfidf.columns:
    train_padded[col] = pad_sequences_with_zeros(train_tfidf[col], 1000)

In [7]:
seq_length = None  # Example sequence length
vocab_size = 200  # Example vocabulary size

# Define encoder input
encoder_inputs = Input(shape=(1000, 1))

# Define LSTM encoder layer with ndim=2
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Define decoder input
# Corrected decoder input shape
decoder_inputs = Input(shape=(1000, 1))

# Define LSTM decoder layer with ndim=2
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(
    decoder_inputs, initial_state=encoder_states)

# Define Dense layer for output
decoder_dense = Dense(1, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy",
              metrics=["accuracy"])

# Print the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1000, 1)]            0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 1000, 1)]            0         []                            
                                                                                                  
 lstm (LSTM)                 [(None, 256),                264192    ['input_1[0][0]']             
                              (None, 256),                                                        
                              (None, 256)]                                                        
                                                                                              

### seq2seq model

In [3]:
story_input = Input(shape=(1000, 1))
question_input = Input(shape=(1000, 1))

# Encoder LSTM
encoder = LSTM(512, return_sequences=True)
encoder_outputs = encoder(story_input)

# Attention mechanism
# Attention mechanism takes encoder outputs and decoder LSTM hidden states to compute the context vector
attention = Attention()
context_vector = attention([encoder_outputs, encoder_outputs])  # Use encoder outputs for both keys and values

# Decoder LSTM
# Decoder LSTM takes the context vector as input and generates output sequences
decoder_lstm = LSTM(512, return_sequences=True)
decoder_outputs = decoder_lstm(context_vector)

# Output layer
# Dense layer maps decoder outputs to vocabulary size and applies softmax activation
decoder_dense = Dense(1, activation="softmax")
output = decoder_dense(decoder_outputs)

# Define the model
model = Model([story_input, question_input], output)

# Compile the model
# Compile the model using Adam optimizer and categorical cross-entropy loss
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [9]:
model.fit(
    [x_stories, x_questions],
    y_answers,
    epochs=1,
    validation_split=0.2,
)

250/250 [==============================] - ETA: 0s - loss: 20778.2637 - accuracy: 1.0850e-04 

W0000 00:00:1710708681.350192   54539 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "AuthenticAMD" model: "241" frequency: 2199 num_cores: 4 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 524288 l3_cache_size: 67108864 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


250/250 [==============================] - 8205s 33s/step - loss: 20778.2637 - accuracy: 1.0850e-04 - val_loss: 21719.5645 - val_accuracy: 1.0550e-04


In [5]:
question_sequences = np.vstack(train_df["question"].to_numpy())
story_sequences = np.vstack(train_df["story"].to_numpy())
answers_sequences = np.vstack(train_df["answers"].to_numpy())

In [8]:
stories = train_df['story'].head(10000).values
questions = train_df['question'].head(10000).values
answers = train_df['answers'].head(10000).values

x_stories = np.array(stories)
x_questions = np.array(questions)
y_answers = np.array(answers)

stories = None
questions = None
answers = None

x_stories = np.vstack(x_stories)
x_questions = np.vstack(x_questions)
y_answers = np.vstack(y_answers)

train_df = None


In [14]:
stories = test_df['story'].head(1).values
questions = test_df['question'].head(1).values
answers = test_df['answers'].head(1).values

x_stories = np.array(stories)
x_questions = np.array(questions)
y_answers = np.array(answers)

stories = None
questions = None
answers = None

x_stories = np.vstack(x_stories)
x_questions = np.vstack(x_questions)
y_answers = np.vstack(y_answers)

test_df = None


In [18]:
predictions = model.predict([x_stories, x_questions])

1/1 [==============================] - 1s 1s/step


In [27]:
y_answers

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [ ]:
predicted_texts = []
for prediction in predictions:
    predicted_text = inverse_transform(prediction)
    predicted_texts.append(predicted_text)



In [23]:

from sklearn.feature_extraction.text import TfidfVectorizer

def inverse_transform(tfidf_sequence):
    tfidf_matrix = tfidf_sequence.reshape(1, -1)  # Reshape to a 2D matrix

    # Inverse transform TF-IDF sequence to text using the TF-IDF vectorizer
    text_sequence = TfidfVectorizer.inverse_transform(tfidf_sequence)
    
    # Remove padding and join tokens into sentences
    text_sequence = [token for token in text_sequence if token != 0]  # Assuming 0 represents padding
    text = " ".join(text_sequence)
    
    return text


In [ ]:
cm = confusion_matrix(prediction, tst)

# Identify misclassifications with counts greater than 1
misclassified_indices = np.where(cm > 0)

# Display the confusion matrix and its shape
# print("Confusion Matrix:")
# print(cm)
print("Shape of Confusion Matrix:", cm.shape)

# Plotting the confusion matrix
plt.figure(figsize=(8, 6))

# Define colors for the plot
cmap = plt.cm.Blues
# Set the color for misclassifications
#cmap.set_under('yellow')

plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0.1)
plt.title('Confusion Matrix')
plt.colorbar()

# Adding axis labels 
classes = np.unique(tst)
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes)
plt.yticks(tick_marks, classes)

# Adding annotations for misclassifications with counts > 1
thresh = cm.max() / 2.0
for i, j in zip(*misclassified_indices):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.show()

In [ ]:
story_sequences = story_sequences.reshape(
    (story_sequences.shape[0], story_sequences.shape[1], 1)
)
question_sequences = question_sequences.reshape(
    (question_sequences.shape[0], question_sequences.shape[1], 1)
)
answers_sequences = answers_sequences.reshape(
    (answers_sequences.shape[0], answers_sequences.shape[1], 1)
)

#### get vocabulary size


In [ ]:
from collections import Counter

# Example DataFrame

# Combine all text data from all columns into a single string
all_text = " ".join(test_df.stack().astype(str))

# Tokenize the text into words
words = all_text.split()

# Count the frequency of each word
word_counts = Counter(words)

# Calculate the vocabulary size
vocab_size = len(word_counts)

print("Vocabulary Size:", vocab_size)

### save processed data


In [40]:
data_array = train_padded.to_numpy()
columns = train_df.columns
np.savez_compressed(
    "../datasets/COQA-Conversation/COQA_train_final.npz",
    data=data_array,
    columns=columns,
)

data_array = test_padded.to_numpy()
columns = test_df.columns
np.savez_compressed(
    "../datasets/COQA-Conversation/COQA_test_final.npz",
    data=data_array,
    columns=columns,
)

In [42]:
test_df = None
train_df = None

test_tfidf = None
train_tfidf = None